In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import kagglehub
import math
import time
import gc

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [3]:
print("Downloading dataset...")
path = kagglehub.dataset_download("bhaveshmittal/melanoma-cancer-dataset")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'melanoma-cancer-dataset' dataset.
Path to dataset files: /kaggle/input/melanoma-cancer-dataset


In [4]:
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
BATCH_SIZE = 32
INPUT_SHAPE = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.4
)

# Load Data
train_generator = train_datagen.flow_from_directory(
    path + '/train',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    path + '/train',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED
)

Found 7128 images belonging to 2 classes.
Found 4751 images belonging to 2 classes.


In [6]:
def evaluate_model(
    filters1, filters2, dropout1, dropout2,
    learning_rate, activation, last_activation,
    keep_model=False
):
    model = models.Sequential([
        layers.Conv2D(filters=filters1, kernel_size=(3, 3), activation=activation,
                      input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(filters=filters2, kernel_size=(3, 3), activation=activation),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(filters=128, kernel_size=(3, 3), activation=activation),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(units=128, activation=activation),
        layers.Dropout(dropout1),
        layers.Dense(units=64, activation=activation),
        layers.Dropout(dropout2),
        layers.Dense(units=1, activation=last_activation)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=4,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        verbose=0
    )

    val_acc = history.history["val_accuracy"][-1]

    # ===== MEMORY SAFE CLEANUP =====
    if not keep_model:
        tf.keras.backend.clear_session()
        del model
        del history
        del optimizer
        return val_acc, None, None
    # ===============================

    return val_acc, model, history

In [7]:
search_space = {}
search_space['filters1'] = [32, 64, 128]
search_space['filters2'] = [32,64,128]
search_space['dropout1'] = [0.1, 0.3, 0.5, 0.7]
search_space['dropout2'] = [0.1, 0.3, 0.5, 0.7]
search_space['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
search_space['activation'] = ['relu', 'elu', 'gelu']
search_space['last_activation'] = ['sigmoid']

In [8]:
def firefly_search(n=5, max_iter=5, alpha=0.2, beta0=1, gamma=1):
    print(f"  -> Firefly Start (n={n}, Iter={max_iter}, alpha={alpha}, beta0={beta0}, gamma={gamma})")

    # ----- Bounds derived from search space -----
    lb = np.array([
        min(search_space['filters1']),
        min(search_space['filters2']),
        min(search_space['dropout1']),
        min(search_space['dropout2']),
        min(search_space['learning_rate'])
    ])

    ub = np.array([
        max(search_space['filters1']),
        max(search_space['filters2']),
        max(search_space['dropout1']),
        max(search_space['dropout2']),
        max(search_space['learning_rate'])
    ])

    # ----- Initialize fireflies (continuous) -----
    fireflies = np.zeros((n, 5))
    for d in range(5):
        fireflies[:, d] = np.random.uniform(lb[d], ub[d], n)

    fitness = np.zeros(n)

    # ----- Initial Evaluation -----
    for k in range(n):
        state = (
            int(round(fireflies[k, 0])),     # filters1
            int(round(fireflies[k, 1])),     # filters2
            fireflies[k, 2],                  # dropout1
            fireflies[k, 3],                  # dropout2
            fireflies[k, 4],                  # learning_rate
            'relu',                           # activation (FIXED)
            'sigmoid'                         # last_activation (FIXED)
        )

        print(
            f"    Evaluating Initial Firefly {k+1}/{n}: "
            f"F1={state[0]}, F2={state[1]}, "
            f"D1={state[2]:.2f}, D2={state[3]:.2f}, LR={state[4]:.5f}"
        )

        acc, _, _ = evaluate_model(*state, keep_model=False)
        fitness[k] = acc

        print(f"      -> Accuracy: {acc:.4f}")

        tf.keras.backend.clear_session()
        gc.collect()

    # ----- Main Loop -----
    for t in range(max_iter):
        print(f"\n--- Iteration {t+1}/{max_iter} ---")

        for i in range(n):
            for j in range(n):
                if fitness[j] > fitness[i]:
                    r = np.linalg.norm(fireflies[i] - fireflies[j])
                    beta = beta0 * np.exp(-gamma * r**2)
                    zeta = np.random.normal(0, 1, size=5)

                    fireflies[i] += beta * (fireflies[j] - fireflies[i]) + alpha * zeta
                    fireflies[i] = np.clip(fireflies[i], lb, ub)

                    state = (
                        int(round(fireflies[i, 0])),
                        int(round(fireflies[i, 1])),
                        fireflies[i, 2],
                        fireflies[i, 3],
                        fireflies[i, 4],
                        'relu',
                        'sigmoid'
                    )

                    print(
                        f"    Firefly {i} moved towards {j}. "
                        f"Evaluating new position..."
                    )

                    acc, _, _ = evaluate_model(*state, keep_model=False)
                    fitness[i] = acc

                    print(f"      -> New Accuracy: {acc:.4f}")

                    tf.keras.backend.clear_session()
                    gc.collect()

    best_index = int(np.argmax(fitness))

    best_state = (
        int(round(fireflies[best_index, 0])),
        int(round(fireflies[best_index, 1])),
        fireflies[best_index, 2],
        fireflies[best_index, 3],
        fireflies[best_index, 4],
        'relu',
        'sigmoid'
    )

    print(f"\n  -> Firefly Finished. Best Accuracy: {fitness[best_index]:.4f}")

    return best_state, fitness[best_index]

In [9]:
start_time = time.time()
best_state, best_acc = firefly_search(n=3, max_iter=10, alpha=0.2, beta0=1, gamma=1)
end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Best State: {best_state}")
print(f"Best Accuracy: {best_acc:.4f}")

  -> Firefly Start (n=3, Iter=10, alpha=0.2, beta0=1, gamma=1)
    Evaluating Initial Firefly 1/3: F1=68, F2=89, D1=0.13, D2=0.52, LR=0.08326


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` fu

      -> Accuracy: 0.5285
    Evaluating Initial Firefly 2/3: F1=123, F2=47, D1=0.62, D2=0.11, LR=0.02131
      -> Accuracy: 0.5293
    Evaluating Initial Firefly 3/3: F1=102, F2=47, D1=0.46, D2=0.68, LR=0.01826
      -> Accuracy: 0.5293

--- Iteration 1/10 ---
    Firefly 0 moved towards 1. Evaluating new position...
      -> New Accuracy: 0.7975
    Firefly 1 moved towards 0. Evaluating new position...
      -> New Accuracy: 0.5293
    Firefly 2 moved towards 0. Evaluating new position...
      -> New Accuracy: 0.7918

--- Iteration 2/10 ---
    Firefly 1 moved towards 0. Evaluating new position...
      -> New Accuracy: 0.5287
    Firefly 1 moved towards 2. Evaluating new position...
      -> New Accuracy: 0.5289
    Firefly 2 moved towards 0. Evaluating new position...
      -> New Accuracy: 0.7848

--- Iteration 3/10 ---
    Firefly 1 moved towards 0. Evaluating new position...
      -> New Accuracy: 0.5296
    Firefly 1 moved towards 2. Evaluating new position...
      -> New Acc

In [10]:
test_generator = test_datagen.flow_from_directory(
    path + '/test',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=False,
    seed=SEED
)

Found 1200 images belonging to 2 classes.


In [11]:
filters1, filters2, dropout1, dropout2, learning_rate, activation, last_activation = best_state

optimized_model = models.Sequential([
    layers.Conv2D(filters=filters1, kernel_size=(3, 3), activation=activation, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=filters2, kernel_size=(3, 3), activation=activation),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation=activation),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(units=128, activation=activation),
    layers.Dropout(dropout1),
    layers.Dense(units=64, activation=activation),
    layers.Dropout(dropout2),
    layers.Dense(units=1, activation=last_activation)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optimized_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

history = optimized_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr]
)

test_loss, test_acc = optimized_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

optimized_model.save('FF_optimized_melanoma_model.keras')
print("Final model saved as FF_optimized_melanoma_model.keras")

Epoch 1/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.6663 - loss: 0.5978 - val_accuracy: 0.7367 - val_loss: 0.5218 - learning_rate: 1.0000e-04
Epoch 2/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.8125 - loss: 0.4243 - val_accuracy: 0.7614 - val_loss: 0.4789 - learning_rate: 1.0000e-04
Epoch 3/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 141ms/step - accuracy: 0.8078 - loss: 0.4209 - val_accuracy: 0.8190 - val_loss: 0.4080 - learning_rate: 1.0000e-04
Epoch 4/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.7812 - loss: 0.4206 - val_accuracy: 0.8167 - val_loss: 0.4051 - learning_rate: 1.0000e-04
Epoch 5/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.8346 - loss: 0.3757 - val_accuracy: 0.7796 - val_loss: 0.4398 - learning_rate: 1.0000e-04
Epoch 6/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.8750 - loss: 0.2674 - val_accuracy: 0.7688 - val_loss: 0.4574 - learning_rate: 1.0000e-04
Epoch 7/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms